In [8]:
import os
import mmcv

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import torch.nn.functional as F
import torch
import cv2
import os

In [9]:
config_names = [
    '/opt/ml/segmentation/mmsegmentation/work_dirs/05_upernet_final0/upernet_beit_large_all.py',
    '/opt/ml/segmentation/mmsegmentation/work_dirs/11_segformer_swin_mixed0/11_segformer_swin_large_mixed_aug_all.py',
    '/opt/ml/segmentation/mmsegmentation/work_dirs/12_segformer_swin_all0/12_segformer_swin_large.py',
]
checkpoint_pathes = [
    '/opt/ml/segmentation/mmsegmentation//work_dirs/05_upernet_final0/best_mIoU_epoch_14.pth',
    '/opt/ml/segmentation/mmsegmentation//work_dirs/11_segformer_swin_mixed0/best_mIoU_epoch_18.pth',
    '/opt/ml/segmentation/mmsegmentation//work_dirs/12_segformer_swin_all0/best_mIoU_epoch_18.pth'
    ]


In [10]:
cfg = Config.fromfile(config_names[0])
# config file 들고오기
root= '/opt/ml/segmentation/input/data/mmseg/test'

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = [(448, 448), (512, 512), (576, 576), 
                                          (640, 640), (704, 704), (768, 768)]
cfg.data.test.pipeline[1]['flip'] = True
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4


dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2021-11-04 09:35:49,358 - mmseg - INFO - Loaded 819 images


In [11]:
models = []
for config_name, checkpoint_path in zip(config_names, checkpoint_pathes):
    cfg = Config.fromfile(config_name)
    model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
    checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

    model.CLASSES = dataset.CLASSES
    model.eval();
    model = model.to('cuda')
    models.append(model)

Use load_from_local loader
Use load_from_local loader


In [12]:
from tqdm import tqdm
import torch.nn.functional as F
import torch
import cv2
import os
import albumentations as A

size = 256
transform = A.Compose([A.Resize(size, size)])
print('Start prediction.')

file_name_list = []
preds_array = np.empty((0, size*size), dtype=np.long)


with torch.no_grad():
    for step, data in enumerate(tqdm(data_loader)):
        softs = []
        for model_i , model in enumerate(models):
            if model_i == 0 :
                imgs = [d.to('cuda') for d in data['img'][:2]]
                img_metas = [d.data[0] for d in data['img_metas'][:2]]
            else :
                imgs = [d.to('cuda') for d in data['img']]
                img_metas = [d.data[0] for d in data['img_metas']]
                    
            logit = model(imgs,img_metas,return_loss=False)
            soft = F.softmax(logit, dim=1).detach().cpu().numpy().astype(np.float16)
            softs.append(soft)
        soft_sum = sum(softs)
        oms = np.argmax(soft_sum,1)
        
        # resize (256 x 256)
        temp_mask = []
        for img, mask in zip(np.stack(imgs[0].detach().cpu().numpy()), oms):
            transformed = transform(image=img, mask=mask)
            mask = transformed['mask']
            temp_mask.append(mask)

        oms = np.array(temp_mask)

        oms = oms.reshape([oms.shape[0], size*size]).astype(int)
        preds_array = np.vstack((preds_array, oms))

print("End prediction.")

/opt/conda/envs/segmentation/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]
  0%|          | 0/819 [00:00<?, ?it/s]

Start prediction.


100%|██████████| 819/819 [16:33<00:00,  1.30s/it]

End prediction.


In [13]:
# sample_submisson.csv 열기
submission = pd.DataFrame({"image_id":[],"PredictionString":[]})
json_dir = os.path.join("../input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# PredictionString 대입
for image_id, predict in enumerate(preds_array):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in predict.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('/opt/ml/segmentation', 'ensemble.csv'), index=False)